<div align="center">
    <h1>Variational Inference and Generative Modelling Tutorial</h1>
    <h3> Weizmann AI Hub for Scientific Discovery </h3>
    <h4>Nathan LEVY</h4>
    nathan.levy@weizmann.ac.il
    <p>with inputs from N.Yosef, M. Kim</p>
    <p>August 15th</p>
</div>




In this tutorial, we are going to delve into a class of generative models called Variational Auto-Encoders. We start by stating key statistical concepts before deriving the relevant mathematical formulation. Then we build and train a VAE. \
Finally, we are going to apply the models we built to single-cell RNA-seq data, and show how these models can be used for representation learning.

💡 This tutorial does not assume prior knowledge in variational inference or biology, but does rely on basic deep learning skills, such as building and training a feedforward neural network. 

# Introduction

To begin with, let us review a few useful ML concepts. 

### Discriminative versus Generative learning


**Discriminative Learning:** \
Discriminative learning is a machine learning approach that focuses on learning the boundary or decision boundary that separates different classes or categories within the data. The primary goal of discriminative models is to find a function that directly maps the input data to the corresponding class labels. These models are designed to distinguish between different classes based on their distinct features or patterns.

A basic example of discriminative model is logistic regression, it is often used in classification tasks where the goal is to assign input data points to specific predefined categories or classes.



**Generative Learning:**\
Generative learning, on the other hand, is a machine learning approach that aims to model the underlying probability distribution of the entire dataset. Instead of focusing solely on learning the decision boundary, generative models attempt to capture the inherent structure and patterns within the data. This allows them to generate new samples that resemble the original data distribution.

Generative models can be used for various tasks, including data synthesis, image generation, and anomaly detection. Some popular generative models include Gaussian Mixture Models (GMMs), Variational Autoencoders (VAEs), and Generative Adversarial Networks (GANs).

In summary, discriminative learning only models the outputs given the inputs, while generative learning models the joint probability of inputs and outputs. 


The focus of this tutorial is on generative learning. To fit the generative models, we adopt the Bayesian inference paradigm.

### Bayesian modelling and Variational Inference


Suppose we have a probabilistic model with some parameters we want to estimate. The Bayesian paradigm involves the following key concepts:

1. **Prior Probability:** Before observing any new data, we have an initial belief about our model parameters $\theta$. This belief is expressed as the prior probability $p(\theta)$.

2. **Likelihood:** Given a dataset $D$, the likelihood function $L$ describes how likely the data is to occur given the underlying parameters: $L=p(D \mid \theta)$

3. **Posterior Probability:** After observing new data, the Bayesian approach updates the initial beliefs to incorporate information from this data. This process is called *Bayesian inference*. The updated parameters are represented as the posterior probability $p(\theta \mid D)$, which combines the prior probability and the likelihood.

4. **Evidence:** The term $p(D)$ represents the likelihood of observing the data across all possible values of the parameters or hypotheses. It acts as a normalizing constant to ensure that the posterior probability integrates to 1. The main goal of any Bayesian model is to optimize the evidence for the data, which is also known as the marginal likelihood. It is often more convenient to work with the log marginal likelihood.


The posterior probability is computed via the *Bayes rule*:

$$ p(\theta \mid D) = \frac{p(D|θ)p(θ)}{\int p(D|θ)p(θ)dθ} = \frac{p(D|θ)p(θ)}{p(D)}$$


However, in most models, the posterior distribution is intractable because of the integral defining the evidence, which runs over the space of all $\theta$. Therefore, the posterior must be estimated: this is called *approximate Bayesian inference*. 

There are a few popular approximate Bayesian inference methods, among which Markov Chain Monte Carlo (the idea is to construct a Markov chain that has the posterior distribution as its equilibrium distribution) and Variational Inference, which tries to minimize the distance between an approximate, tractable posterior (called the variational distribution) and the true one. We are going to focus on the latter. 


### Latent Variable Models

Given observed data $D$, Latent Variable Models assume that the data generating process is driven by unobserved *latent variables* $Z$, of smaller dimension. The latent variables should be a summary of the data, driving its variability. They can represent a useful representation of the data. 


Mixture Models are a classsic example of LVMs. They assume that the dataset $D$ is generated by sampling i.i.d. from $K$ distincts distributions called mixture components. We can describe the data generating process as follows: 

1. For each datapoint, we first sample the mixture assignment from a categorical random variable, out of $K$ possible components. 
2. Then, we sample from the assigned distribution. 

In this model, we do observe the datapoints, which were drawn conditionnally on the mixture assignment, but we do not observe the assignment itself: it is the latent variable. 

Given the observed data, we want to infer the latent variables, using the posterior distribution $p(Z \mid D)$. For this purpose, we'll have to rely on approximate Bayesian inference, more precisely Variational Inference. Thus we introduce the approximate posterior $q(Z \mid D)$, which comes from a family of tractable distributions. As we said, the goal is to minimize the distance between the true and approximate posterior. The common metric to use is the [Kullback-Leibler divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence). It is defined as: 
$$\mathbb{KL}(q \mid p) = \int q(x) \log \frac{q(x)}{p(x)} dx$$

The Kullback-Leibler divergence is a measure of how one probability distribution diverges from a second, expected probability distribution. In other words, it is a measure of how much information is lost when the second distribution is used to approximate the first one.

<div class="alert alert-block alert-info">
<b>Question 1</b>
Is the KL divergence symmetric? What is its value when the two distributions are equal? What is its minimum value?
</div>

### Evidence Lower Bound (ELBO)

We want to fit out latent variable model to the data, but the log marginal likelihood is intractable. We also want to evaluate the posterior distribution $p(Z \mid D)$, but it is also intractable, as its denominator is the evidence. Instead, we are going to maximize a lower bound on the log marginal likelihood, called the Evidence Lower Bound (ELBO).

Recall from Bayes rule that the log marginal likelihood can be written as:
$$\log p(D) = \log\frac{p(D \mid Z)p(Z)}{p(Z \mid D)} = \mathbb{E}_{Z \sim q} \left[ \log\frac{p(D \mid Z)p(Z)}{p(Z \mid D)} \right]$$

Taking the expectation with respect to the approximate posterior $q(Z \mid D)$, we get:

$$\log \left[ p(D) \right] = \mathbb{E}_{Z \sim q} \left[ \log\frac{p(D \mid Z)p(Z)}{p(Z \mid D)} \right]$$

Multiplying and dividing by $q(Z \mid D)$, we get:

$$\log \left[ p(D) \right] = \mathbb{E}_{Z \sim q} \left[ \log\frac{p(D \mid Z)p(Z)}{p(Z \mid D)} \frac {q(Z|D)}{q(Z|D)} \right]$$

Thus we can decompose the log marginal likelihood as follows:

$$\log p(D) = \mathbb{E}_{Z \sim q} \log \frac{q(Z|D)}{p(Z|D)} + \mathbb{E}_{Z \sim q} \log \frac{p(Z)}{q(Z|D)} + \mathbb{E}_{Z \sim q} \log p(D | Z)$$

This can be re-expressed using the KL divergence as follows:

$$\log p(D) = \mathbb{KL}(q(Z | D) \| p(Z | D)) - \mathbb{KL}(q(Z | D) \| p(Z)) + \mathbb{E}_{Z \sim q} \log p(D|Z)$$

Because the KL divergence is always positive, we get the following inequality:
$$\log p(D) \geq - \mathbb{KL}(q(Z | D) \| p(Z)) + \mathbb{E}_{Z \sim q} \log p(D|Z)$$

The right-hand side of the inequality is called the Evidence Lower Bound [(ELBO)](https://en.wikipedia.org/wiki/Evidence_lower_bound). It is a lower bound on the log marginal likelihood. By maximizing the ELBO, we avoid the intractability of the log marginal likelihood. 

$$ELBO =  - \mathbb{KL}(q(Z | D) \| p(Z)) + \mathbb{E}_{Z \sim q} \log p(D|Z)$$

The ELBO includes two sets of parameters: the parameters  $\phi$ of the approximate posterior $q$ and the parameters  $\theta$ of likelihood $p$.

### Deep Generative Models (DGMs)

Deep generative models are generative models that assume that both distributions $q$ and $p$ are parameterized by neural networks.
Variational Autoencoders (VAEs) are a class of DGMs with two key components: an encoder and a decoder. The encoder maps the input data to a latent representation, while the decoder maps the latent representation back to the input space. The encoder $f_\phi$ and decoder $f_\theta$ are trained jointly to maximize the ELBO.

A typical choice for the latent prior $p(Z)$ is the standard isotropic Gaussian $\mathcal{N}(0,I)$, while the appoximate posterior can be chosen as diagonal Gaussian
$$q(Z \mid D) = \mathcal{N}(\mu_\phi(D), \sigma_\phi(D))$$

such that:

$$\mu_\phi(D), \sigma_\phi(D) = f_\phi(D)$$

Where $f_\phi$ is a neural network with parameters $\phi$ and $\sigma_\phi(D)$ is a diagonal covariance matrix with zero off-diagonal elements.

With these choices, the KL divergence term in the ELBO can be computed in closed form.
The form of decoder distribution changes with the type of data we want to model. For example, if we want to model images, we can use a Bernoulli distribution for the decoder. If we want to model continuous data, we can use a (diagonal) Gaussian distribution for the decoder, such that:
$$p(D \mid Z) = \mathcal{N}(\mu_\theta(Z), \sigma_\theta(Z))$$
With $\mu_\theta(Z), \sigma_\theta(Z) = f_\theta(Z)$.


Using the 3 distributions above, we can analytically compute the two ELBO terms: the KL $\mathbb{KL}(q(Z | D) \| p(Z))$ and the reconstruction term $\mathbb{E}_{Z \sim q} \log p(D|Z)$.

<div class="alert alert-block alert-info">
<b>Question 2</b>
What is the analytical expression of the KL divergence? What is the analytical expression of the reconstruction term for a Gaussian decoder?
</div>

# Design and train a VAE

Now that we derived the mathematical formulation of the VAE, we are going to implement it. We will use the MNIST dataset, which consists of 28x28 grayscale images of handwritten digits. The goal is to train a VAE to generate new images of handwritten digits.

# Applying DGMs to sc-RNA-seq data